In [ ]:
import weaviate
import json
import pprint

print("Let's a go!")

print("Connecting to weaviate instance on localhost:8080...")
client = weaviate.Client("http://localhost:8080")
print("Client created")

pprint.pprint(client.schema.get())

result = (
    client.query
    .get("DriftBenchmark", ["prompt", "completion", "date_time", "chat_id"])
    .do()
)
print(json.dumps(result, indent=4))

In [ ]:
import weaviate
import json

print("Let's a go!")

print("Connecting to weaviate instance on localhost:8080...")
client = weaviate.Client("http://localhost:8080")
print("Client created")

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
print("sbert loaded")

def process_query(vec):
    where_filter = {
        "path": ["chat_id"],
        "operator": "Equal",
        "valueText": "c5f83121-758e-46b2-99d1-5ca0c6f59919",
    }
    nearVector = {"vector": vec}
    res = client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion", "_additional {certainty, distance}"]).with_near_vector(nearVector).with_where(where_filter).do()
    print("------------------------------------------------------------------------------------------------")
    print("-----------------------------------Most similar text -------------------------------------------")
    print("ChatID: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['chat_id'], end='')
    print(", Prompt: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt'], end='')
    print(", Completion: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['completion'], end='')
    print(", Certainty: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['_additional']['certainty'])
    print(", Distance: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['_additional']['distance'])
    print("------------------------------------------------------------------------------------------------")

print("------------------------------------------------------------------------------------------------")
print("Running a test query first...")

# this is exactly the text in cmpletion for 
#{
#"prompt": "how to create a custom shell using click and cmd",
#"completion": "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to",
#"chat_id": "c5f83121-758e-46b2-99d1-5ca0c6f59919"
#}

test_query = "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to"
query_vec = sbert_model.encode(test_query)
print(query_vec)
process_query(query_vec)
print("------------------------------------------------------------------------------------------------")
# Certainty: 0.9343259930610657

# now shorten it by 20 chars
test_query = "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific comma"
query_vec = sbert_model.encode(test_query)
print(query_vec)
process_query(query_vec)
print("------------------------------------------------------------------------------------------------")
# 1 Certainty: 0.9343259930610657
# 2 Certainty: 0.9157571196556091
#  difference: 0.0185688734054566

# 1 Distance: 0.13134801
# 2 Distance: 0.16848576
# difference: 0.03713775

# while True:
    
#     query = input("Enter query:")
#     if query=='q':
#         break

#     query_vec = sbert_model.encode(query)
#     #print(query_vec)
#     process_query(query_vec)

# Query with Cursor

In [ ]:
import weaviate
import json

print("Let's a go!")

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

limit = 10
result_count = 0
aggregate_count = 0
batch_size = 5
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

def wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        print("{}. {}".format(result_count, res['completion']))
        # res["data"]["Get"]
        result_count += 1
        
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]


# Output
```
Let's a go!
Connecting source_client to weaviate instance on localhost:8080...
source_client created
aggregate_count = 5
0. Subject: Dermacol SF - SMS Marketing Ideas  Segment 1: New Customers  1. 'Hey [Name]! Welcome to Dermacol SF! Get flawless with our best-selling Make-up Cover! Use code NEWBIE15 for 15% off on your first purchase. Shop now: [link]' 2. 'New to Dermacol SF? Discover our Fixing Powder for a perfect, all-day matte finish. Enjoy 15% off your first order with code WELCOME15. Shop now: [link]'  Segment 2: Existing Customers 3. 'Hi [Name]! Thank you for being a loyal Dermacol SF customer. Enjoy 20% off 
1. The NGM SCM150 motor has six wires or cables: three motor phase wires and three hall sensor wires. The colors of these wires may vary depending on the manufacturer, so it is important to refer to the motor's datasheet or manual for specific information.  In general, the motor phase wires are usually color-coded as follows:  * Black or brown wire: U-phase * White or yellow wire: V-phase * Red or blue wire: W-phase  As for the hall sensor wires, they are usually color-coded as follows:  * Green wi
2. Sure, I'd be happy to help you come up with a content schedule for your blog targeting the long tail keyword 'Plan a Golf Trip to Madrid'. Here's a content calendar for the next four weeks:  **MERCHYNT'S MAGICAL CONTENT CALENDAR FOR PLAN A GOLF TRIP TO MADRID**  To learn more about Merchynt's local SEO tools visit [www.merchynt.com-partners](http:--www.merchynt.com-partners).  **Week 1:**  | Date | Title | | --- | --- | | Feb 20 | 'Madrid Golf: The Ultimate Guide for Planning Your Next Golf Trip
3. Sure, here's my presentation on the furniture exports of the Philippines:  ---  Good day everyone,  Today, I will be discussing the current state of furniture exports in the Philippines.  As of 2021, the furniture industry is a growing sector in the country's economy, with a strong focus on exporting products to foreign markets. According to the Philippine Statistics Authority, furniture and fixtures ranked as the fifth top export earner in the country's manufacturing sector, accounting for 3.1%
4. Dear Hina,  Thank you for considering me for the position in procurement and business department at Cgico. I appreciate your interest in my profile and would be glad to answer your question regarding my PEC certificate.  Yes, I do possess a PEC certificate and would be happy to provide any further documentation or details regarding my certification, if required.  I am excited about the opportunity to interview with your organization and would be available for an interview either today or tomorro
aggregate_count = 10
5. Sure, here's a sample itinerary for a 4-day trip with the 20 most recommended places to visit with children near Chiang Mai:  Day 1:  1. Doi Suthep-Pui National Park - This national park is home to Doi Suthep, a mountain peak with a stunning temple on top. The park also offers hiking trails, waterfalls, and scenic views. Address: Tambon Su Thep, Amphoe Mueang Chiang Mai, Chang Wat Chiang Mai 50200, Thailand. Rank in Google: 4.6. Opening Hours: 6:00 AM - 6:00 PM. Entry price for an adult: 30 THB,
6. 1. 'Revolutionize your digital products with our user-centric approach.' 2. 'Elevate your software to the next level with our Agile methodology.' 3. 'Experience faster time-to-market and improved quality with our DevOps solutions.' 4. 'Design Thinking and Agile methodologies come together for a game-changing user experience.' 5. 'Maximize efficiency and reduce risks with our cutting-edge technologies.' 6. 'Create a seamless user experience with our user-centric approach.' 7. 'Empower your digita
7. Of course! Here's a simulation of a job interview for a sales manager position:  Interviewer: Thank you for coming in today. Can you tell us a little bit about your background and experience in sales?  Candidate: Absolutely. I have been working in sales for the past 10 years, with the last five years specifically focused on sales management. In my current role, I manage a team of 10 sales representatives and have consistently exceeded our sales targets.  Interviewer: That's great to hear. Can yo
8. One example of financial donations to monasteries in the Middle Ages can be found in the book 'The Age of Cathedrals: Art and Society, 980-1420' by Georges Duby. He describes how the construction and maintenance of many cathedrals and monasteries in medieval Europe was financed through donations from wealthy patrons, such as kings, nobles, and merchants. These donations were often made in the form of land, money, or other forms of wealth, and were used to support the religious communities and th
9. Here's a simple algorithm to check for overlapping groups:  1. Create an empty list `groups`. 2. For each group, store the range of values in a tuple and add it to the `groups` list. 3. Sort the `groups` list based on the lower bounds of the ranges. 4. Loop through the `groups` list and compare each group's upper bound with the next group's lower bound. 5. If the upper bound of a group is greater than or equal to the lower bound of the next group, then there is an overlap and you should raise an
```



In [ ]:
import weaviate
import cohere

from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('COHERE_API_KEY')

def IsOkString(value):
    if not value or len(value)==0:
        return False 
    else:
        return True

print("Let's a go!")

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

limit = 1
result_count = 0
aggregate_count = 0
batch_size = 3
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

print("Creating cohere_client")
cohere_client = cohere.Client(api_key) # This is your trial API key


def wv_wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()


print("Have fun storming the castle!")

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        
        the_prompt = res['prompt']
        print("{}. completion: {}".format(result_count, res['completion']))
        result_count += 1
        
        response = cohere_client.generate(
            model='command',
            prompt=the_prompt,
            max_tokens=300,
            temperature=0.9,
            k=0,
            stop_sequences=[],
            return_likelihoods='NONE')
        
        # print the full response object:
        # print("Cohere prompt response: {}".format(response))
        
        prompt_response = response.generations[0].text.replace("\n", " ")
        print("\nCohere prompt response:\n{}".format(prompt_response))        
        print("------------------------------------------------------------------------------------------------")
    
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]

# Cursor with Cohere API calls

```
Let's a go!
Connecting source_client to weaviate instance on localhost:8080...
source_client created
Creating cohere_client
Have fun storming the castle!
aggregate_count = 3

0. completion: Subject: Dermacol SF - SMS Marketing Ideas  Segment 1: New Customers  1. 'Hey [Name]! Welcome to Dermacol SF! Get flawless with our best-selling Make-up Cover! Use code NEWBIE15 for 15% off on your first purchase. Shop now: [link]' 2. 'New to Dermacol SF? Discover our Fixing Powder for a perfect, all-day matte finish. Enjoy 15% off your first order with code WELCOME15. Shop now: [link]'  Segment 2: Existing Customers 3. 'Hi [Name]! Thank you for being a loyal Dermacol SF customer. Enjoy 20% off 

Cohere prompt response:
 Hey there! We've got some great news for you. We've just launched a new product that we think you'll love - the Make-up Cover! This product is perfect for those who want to achieve a flawless look. It provides full coverage and a matte finish, and it's available in a variety of shades to match any skin tone. So come on and give it a try!
------------------------------------------------------------------------------------------------
1. completion: The NGM SCM150 motor has six wires or cables: three motor phase wires and three hall sensor wires. The colors of these wires may vary depending on the manufacturer, so it is important to refer to the motor's datasheet or manual for specific information.  In general, the motor phase wires are usually color-coded as follows:  * Black or brown wire: U-phase * White or yellow wire: V-phase * Red or blue wire: W-phase  As for the hall sensor wires, they are usually color-coded as follows:  * Green wi

Cohere prompt response:
 The NGM SCM150 motor has three Hall sensor cables, which are typically green, red, and black. The green cable is connected to the A1 pin on the motor controller, the red cable is connected to the A2 pin, and the black cable is connected to the A3 pin. These cables provide feedback to the motor controller about the position of the motor.
------------------------------------------------------------------------------------------------
2. completion: Sure, I'd be happy to help you come up with a content schedule for your blog targeting the long tail keyword 'Plan a Golf Trip to Madrid'. Here's a content calendar for the next four weeks:  **MERCHYNT'S MAGICAL CONTENT CALENDAR FOR PLAN A GOLF TRIP TO MADRID**  To learn more about Merchynt's local SEO tools visit [www.merchynt.com-partners](http:--www.merchynt.com-partners).  **Week 1:**  | Date | Title | | --- | --- | | Feb 20 | 'Madrid Golf: The Ultimate Guide for Planning Your Next Golf Trip

Cohere prompt response:
 If you're looking for a destination to plan a golf trip, Madrid should be at the top of your list! With a variety of courses to choose from, you're sure to find one that suits your skill level and interests.  One of the most popular courses in Madrid is the Club de Golf Retiro, which is located just outside the city center. This course is known for its challenging layout and beautiful surroundings, with plenty of trees and water hazards to keep you on your toes.  Another great course in Madrid is the Real Club de Golf El Prat, which is located just outside the city as well. This course is known for its pristine conditions and friendly staff, making it a great choice for beginners and experienced golfers alike.  If you're looking for a more budget-friendly option, the Hesperia Madrid Hotel & Golf is a great choice. This course is located just outside the city and offers a variety of discounts and promotions throughout the year.  So whether you're looking for a challenging course or a more relaxed round of golf, Madrid has a course for you!
------------------------------------------------------------------------------------------------
```


## Comparing Sentence Similarities

The sentences (texts) are mapped such that sentences with similar meanings are close in vector space. One common method to measure the similarity in vector space is to use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). For two sentences, this can be done like this:

Note: the model we'll (multi-qa-MiniLM-L6-cos-v1) use must be the same one we used to create the original DriftBenchmark import.

```python
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to")
emb2 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific comma")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)
```

In [ ]:
# try it out before adding to the cursor query

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to")
emb2 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific comma")

cos_sim = util.cos_sim(emb1, emb2)
print(type(cos_sim))
print("Cosine-Similarity:", cos_sim)
print("Cosine-Similarity tensor:", cos_sim[0][0])
print("Cosine-Similarity tensor:", cos_sim.item())

Now we add the cosine_similarity calculation

In [ ]:
import weaviate
import cohere
from sentence_transformers import SentenceTransformer, util

def IsOkString(value):
    if not value or len(value)==0:
        return False 
    else:
        return True

print("Let's a go!")

print("Load the sentence transformer model: multi-qa-MiniLM-L6-cos-v1 ")
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')


print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

limit = 1
result_count = 0
aggregate_count = 0
batch_size = 3
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

print("Creating cohere_client")
cohere_client = cohere.Client(api_key) # This is your trial API key


def wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()


print("Have fun storming the castle!")

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        the_prompt = res['prompt']
        result_count += 1
        
        response = cohere_client.generate(
            model='command',
            prompt=the_prompt,
            max_tokens=300,
            temperature=0.9,
            k=0,
            stop_sequences=[],
            return_likelihoods='NONE')
                
        completion = response.generations[0].text.replace("\n", " ")
        vec_current_completion = model.encode(completion)
        cos_sim = util.cos_sim(res['_additional']['vector'], vec_current_completion)
        print("Cosine-Similarity:", cos_sim.item())
        print("------------------------------------------------------------------------------------------------")
    
    
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]

# Add some rows to sqlite db
```sql

sqlite3 database: driftDb

INSERT INTO tSample (
                        chat_id,
                        sample_dt,
                        cos_sim,
                        completion
                    )
                    VALUES (
                        'chat_id',
                        'sample_dt',
                        'cos_sim',
                        'completion'
                    );
```


# Load the baseline samples into sqlite db

Set these as appropriate
5 and 10 is good for dev
```python
batch_size = 5  
limit = 10
```
20 and 1000 is good when ur ready to go
should only need 20 seconds.
```python
batch_size = 20  
limit = 1000
```

In [ ]:
import weaviate
import sqlite3
import datetime
from datetime import timezone
print("At {}, Mario said, Let's a go!".format(datetime.datetime.now(timezone.utc).astimezone().isoformat()))

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

print("Connecting to sqlite driftDb")
DRIFT_DB_PATH = "..\\data\\driftDb.db"
con = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = con.cursor()

run_time = ct = datetime.datetime.now().replace(second=0, microsecond=0)
# sql
def sql_insert_sample(chat_id, cos_sim, completion):
    sql = "INSERT INTO tSample (chat_id, run_time, cos_sim, completion ) VALUES ('{}', '{}', '{}', \"{}\");".format(chat_id, run_time, cos_sim, completion)
    print(sql)
    sql_cursor.execute(sql)
    con.commit()
    
# sql
def sql_insert_sample_default(chat_id, cos_sim, completion):
    sql = "INSERT INTO tSample (chat_id, run_time, cos_sim, completion ) VALUES ('{}', '{}', '{}', \"{}\");".format(chat_id, run_time, cos_sim, completion)
    print(sql)
    sql_cursor.execute(sql)
    con.commit()    
    
batch_size = 20  
limit = 1000
result_count = 0
aggregate_count = 0

class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

#weaviate
def wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        print("{}. {}".format(result_count, res['completion']))
        # res["data"]["Get"]
        result_count += 1
        
        # insert benchmark baseline samples
        sql_insert_sample_default(res['chat_id'], 1.0, res['completion']) 
        
    if(aggregate_count >= limit):
        break

    # Update the wv cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]